이미지를 증식해서 넣어봅시다!!

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

from keras.models import Sequential,load_model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
np.random.seed(1)

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './images2/train',
        target_size = (300,300),
        batch_size=50,
#         color_mode ='rgb',
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        seed = 42,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './images2/test',
#         classes =[],
        target_size=(300,300),
        batch_size=10,
#         color_mode ='rgb',
        shuffle =True,
        seed = 42,
        class_mode='categorical')


Found 36000 images belonging to 3 classes.
Found 9000 images belonging to 3 classes.


In [3]:
iterations = 50

train_plus = train_datagen.flow_from_directory(
    './images2/train',
    target_size = (300,300),
    batch_size = 10,
    class_mode = 'categorical')

for i in enumerate(range(iterations)):
    img, label = train_plus.next()
#     n_img = len(label)
    
#     base = cv2.cvtColor(img[0], cv2.COLOR_RGB2BGR)  # keras는 RGB, openCV는 BGR이라 변경함
#     for idx in range(n_img - 1):
#         img2 = cv2.cvtColor(img[idx + 1], cv2.COLOR_RGB2BGR)
#         base = np.hstack((base, img2))
#     images.append(base)
 
# img = images[0]
# for idx in range(len(images) - 1):
#     img = np.vstack((img, images[idx + 1]))
# cv2.imshow('result', img)

Found 36000 images belonging to 3 classes.


In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=3,
                 activation='relu',
                 input_shape=(300,300,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(16, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
# model.add(Dense(32, activation='selu'))
# model.add(Dense(2, activation='selu'))
model.add(Dense(3, activation='softmax'))

# 3. 모델 학습과정 설정 및 요약
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=10,
        validation_data=test_generator,
        validation_steps=10)

model.save('model-8')

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=10)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 147, 147, 2)       578       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 73, 73, 2)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 10658)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 31977     
Total params: 33,451
Trainable params: 33,451
Non-trainable params: 0
__________________________________________________